In [1]:
import os
import sys 
import subprocess as sp
import yaml
import glob

In [2]:
def build_containers(yaml_dict):
    
    simg_name = yaml_dict['module_1']['container']['simg_name']
    dockerhub_path = yaml_dict['module_1']['container']['dockerhub_path']

    if not os.path.isfile(simg_name):
        print(f'Building {simg_name}.')
        sp.call(f'singularity build {simg_name} {dockerhub_path}', shell=True)

def download_raw_data(irods_path):
    hpc = None
    file_name = os.path.basename(irods_path)
    dir_name = file_name.split('.')[0]

    if not os.path.isdir(dir_name):
        cmd1 = f'iget -fKPVT {irods_path}'
        print(cmd1)
        cwd = os.getcwd()

        if '.gz' in file_name: 
            cmd2 = f'tar -xzvf {file_name}'
            cmd3 = f'rm {file_name}'

        else: 
            cmd2 = f'tar -xvf {file_name}'
            cmd3 = f'rm {file_name}'
        
        if hpc: 
            print('>>>>>>Using data transfer node.')
            sp.call(f"ssh filexfer 'cd {cwd}' '&& {cmd1}' '&& {cmd2}' '&& {cmd3}' '&& exit'", shell=True)
            
        else: 
            sp.call(cmd1, shell=True)
            sp.call(cmd2, shell=True)
            sp.call(cmd3, shell=True)

    return dir_name

def build_containers(dictionary):
    for k, v in dictionary['modules'].items():
        container = v['container']
        if not os.path.isfile(container["simg_name"]):
            print(f'Building {container["simg_name"]}.')
            sp.call(f'singularity build {container["simg_name"]} {container["dockerhub_path"]}', shell=True)

def get_model_files(model_path):
    
    if not os.path.isfile(os.path.basename(model_path)):
        cmd1 = f'iget -fKPVT {model_path}'
        sp.call(cmd1, shell=True)
    return os.path.basename(model_path)
    

In [6]:
date = '2020-01-22'
with open("./config.yaml", 'r') as stream:
    try:
        dictionary = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

    cyverse_path = os.path.join(dictionary['paths']['cyverse']['input']['basename'], 
                                date,
                                dictionary['paths']['cyverse']['input']['subdir'], 
                                ''.join([str(date), str(dictionary['paths']['cyverse']['input']['suffix'])]))

    dir_name = download_raw_data(cyverse_path)
    build_containers(dictionary)
    model_name = get_model_files('/iplant/home/shared/phytooracle/season_10_lettuce_yr_2020/level_0/necessary_files/dgcnn_3d_model.pth')


    for plant in glob.glob(os.path.join(dir_name, '*'))[:1]:
 
        for k, v in dictionary['modules'].items():
            command = v['command'].replace('${PLANT_NAME}', plant).replace('${MODEL_PATH}', model_name)
            print(command)
            sp.call(command, shell=True)

singularity run -B $(pwd):/mnt --pwd /mnt dgcnn_single_plant_soil_segmentation.simg -i 2020-01-22_test_set/Iceberg_114 --model_path dgcnn_3d_model.pth
singularity run -B $(pwd):/mnt --pwd /mnt 3d_neighbor_removal.simg -i 2020-01-22_test_set/Iceberg_114 -e 0.03
singularity run -B $(pwd):/mnt --pwd /mnt 3d_entropy_features.simg -o combined_pointclouds/2020-01-22_test_set/Iceberg_114 -f 2020-01-22_test_set/Iceberg_114_hull_volumes -p individual_plants_out/combined_pointclouds/2020-01-22_test_set/Iceberg_114/*_plant_clustered.ply
